# Applied Data Science Capstone Project

IBM Data Sceince Professional Certificate Specialization

In [1]:
import pandas as pd
import numpy as np

#### Get Wikipedia page's content.
Use requests package to grap page's content.

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

try:
    response = requests.get(url)
except requests.exceptions.RequestException as e:
    raise SystemExit(e)
    
if response.status_code != 200:
    raise SystemExit(e)

page = response.content

#### Let's use BeautifulSoup to scrap the data
* Parse content with BeautifulSoup
* Pre-precess data and create a dataframe.

credits for hints to https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table

In [10]:
soup = BeautifulSoup(page)
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
table = soup.find('table', attrs={'class':'wikitable'}).find('tbody')

for row in table.find_all('tr'):
    row = row.find_all('td')
    row = [e.text.strip() for e in row]

    # process cells with assigned borough only
    if row == [] or row[1] == 'Not assigned':
        continue
        
    # replace backslashes with commas   
    row[2] = row[2].replace(' /', ',')
    
    # Neighborhood should be the same as the boroughNot if empty
    if row[2] is None:
        row[2] = row[1]
    
    df = df.append(pd.Series(row, index = df.columns), ignore_index=True)
    
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check shape of the dataframe

In [4]:
df.shape

(103, 3)

#### Getting the latitude and the longitude coordinates of each neighborhood.
Use provided in project description csv file **Geospatial_data** to obtain coordinates for postal code.
* Load geospatial data.

In [8]:
geospatial_url = 'https://cocl.us/Geospatial_data'
geo_data = pd.read_csv(geospatial_url)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


* Extend dataframe with latitude and longitude coordinates.

In [13]:
df = df.join(geo_data.set_index('Postal Code'), on = 'PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
